# Resumen de Resultados: CT Filtrado

Este notebook documenta y resume todos los resultados del análisis de **Cambio de Tiempo Filtrado (CT_Filtrado)**.

## Objetivo:

Mostrar claramente:
1. Qué ítems CT individuales fueron significativos
2. Qué variables CT_Filtrado resultaron significativas
3. Estadísticas descriptivas por categoría
4. Comparación con CO_Filtrado
5. Tablas Excel con todos los resultados

In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats import kruskal
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

print("✓ Librerías cargadas exitosamente")

✓ Librerías cargadas exitosamente


## 1. Cargar Datos y Ejecutar Notebooks Previos

In [2]:
%%capture
%run "38. Comparaciones estadísticas de nuevos CT filtrados.ipynb"

In [3]:
print("✓ Datos cargados exitosamente")
print(f"  DataFrames disponibles: {list(dfs_Finales.keys())}")

✓ Datos cargados exitosamente
  DataFrames disponibles: ['Generales', 'Ballotage']


## 2. Ítems CT Individuales Significativos

Estos son los ítems que mostraron diferencias significativas entre categorías en el test de Kruskal-Wallis.

In [4]:
print("="*70)
print("ÍTEMS CT INDIVIDUALES SIGNIFICATIVOS")
print("="*70)

for Nombre_df in dfs_Finales.keys():
    print(f"\n📊 {Nombre_df}:")
    
    Items_Sig = Todas_Variables_Significativas['Cambios_Tiempo_Por_Item_Significativos'][Nombre_df]
    
    if Items_Sig:
        print(f"   Total de ítems significativos: {len(Items_Sig)}")
        print(f"\n   Variables significativas:")
        for item in Items_Sig:
            print(f"   - {item}")
    else:
        print("   No hay ítems significativos")

print("\n" + "="*70)

ÍTEMS CT INDIVIDUALES SIGNIFICATIVOS

📊 Generales:
   Total de ítems significativos: 14

   Variables significativas:
   - CT_Item_7_Der
   - CT_Item_4_Der
   - CT_Item_30_Izq
   - CT_Item_16_Der
   - CT_Item_16_Izq
   - CT_Item_25_Der
   - CT_Item_10_Der
   - CT_Item_7_Izq
   - CT_Item_3_Der
   - CT_Item_22_Izq
   - CT_Item_23_Der
   - CT_Item_3_Izq
   - CT_Item_9_Der
   - CT_Item_4_Izq

📊 Ballotage:
   Total de ítems significativos: 5

   Variables significativas:
   - CT_Item_30_Izq
   - CT_Item_7_Der
   - CT_Item_8_Der
   - CT_Item_10_Der
   - CT_Item_29_Izq



## 3. Variables CT_Filtrado Creadas

Estas son las variables sumadas que se crearon a partir de los ítems significativos:

In [5]:
Variables_CT_Filtrado = [
    'Cambio_Tiempo_Filt_Pro_Izq',
    'Cambio_Tiempo_Filt_Pro_Der',
    'Cambio_Tiempo_Filt_Con_Izq',
    'Cambio_Tiempo_Filt_Con_Der'
]

print("Variables CT_Filtrado creadas:")
for var in Variables_CT_Filtrado:
    print(f"  - {var}")

Variables CT_Filtrado creadas:
  - Cambio_Tiempo_Filt_Pro_Izq
  - Cambio_Tiempo_Filt_Pro_Der
  - Cambio_Tiempo_Filt_Con_Izq
  - Cambio_Tiempo_Filt_Con_Der


## 4. Resultados de Kruskal-Wallis para CT_Filtrado

Test de significancia entre categorías para cada variable CT_Filtrado:

In [6]:
print("="*70)
print("RESULTADOS KRUSKAL-WALLIS: CT_FILTRADO")
print("="*70)

for Nombre_df in dfs_Finales.keys():
    print(f"\n📊 Dataset: {Nombre_df}")
    print("\n" + "-"*70)
    
    df = dfs_Finales[Nombre_df]
    
    # Filtrar categorías válidas
    Categorias_Validas = [
        'Left_Wing', 'Progressivism', 'Centre',
        'Moderate_Right_A', 'Moderate_Right_B', 'Right_Wing_Libertarian'
    ]
    
    df_filtrado = df[df['Categoria_PASO_2023'].isin(Categorias_Validas)].copy()
    
    # Para cada variable CT_Filtrado
    for var in Variables_CT_Filtrado:
        if var in df_filtrado.columns:
            # Preparar datos
            datos_validos = df_filtrado[['Categoria_PASO_2023', var]].dropna()
            
            if len(datos_validos) > 0:
                # Agrupar por categoría
                grupos = [
                    datos_validos[datos_validos['Categoria_PASO_2023'] == cat][var]
                    for cat in Categorias_Validas
                ]
                
                # Filtrar grupos vacíos
                grupos_validos = [g for g in grupos if len(g) > 0]
                
                if len(grupos_validos) > 1:
                    try:
                        h_stat, p_valor = kruskal(*grupos_validos)
                        
                        # Determinar significancia
                        if p_valor < 0.001:
                            sig = '***'
                        elif p_valor < 0.01:
                            sig = '**'
                        elif p_valor < 0.05:
                            sig = '*'
                        else:
                            sig = 'ns'
                        
                        print(f"\n{var}:")
                        print(f"   Estadístico H: {h_stat:.4f}")
                        print(f"   p-valor: {p_valor:.6f}")
                        print(f"   Significancia: {sig}")
                        
                        if sig != 'ns':
                            print(f"   ✅ VARIABLE SIGNIFICATIVA")
                        else:
                            print(f"   ❌ No significativa")
                            
                    except Exception as e:
                        print(f"\n{var}: Error en cálculo - {e}")
                else:
                    print(f"\n{var}: Insuficientes grupos para análisis")
            else:
                print(f"\n{var}: Sin datos válidos")
        else:
            print(f"\n{var}: Variable no encontrada en dataset")
    
    print("\n" + "-"*70)

print("\n" + "="*70)

RESULTADOS KRUSKAL-WALLIS: CT_FILTRADO

📊 Dataset: Generales

----------------------------------------------------------------------

Cambio_Tiempo_Filt_Pro_Izq:
   Estadístico H: 8.9858
   p-valor: 0.109632
   Significancia: ns
   ❌ No significativa

Cambio_Tiempo_Filt_Pro_Der:
   Estadístico H: 29.1217
   p-valor: 0.000022
   Significancia: ***
   ✅ VARIABLE SIGNIFICATIVA

Cambio_Tiempo_Filt_Con_Izq:
   Estadístico H: 6.4498
   p-valor: 0.264875
   Significancia: ns
   ❌ No significativa

Cambio_Tiempo_Filt_Con_Der:
   Estadístico H: 35.6601
   p-valor: 0.000001
   Significancia: ***
   ✅ VARIABLE SIGNIFICATIVA

----------------------------------------------------------------------

📊 Dataset: Ballotage

----------------------------------------------------------------------

Cambio_Tiempo_Filt_Pro_Izq: Error en cálculo - All numbers are identical in kruskal

Cambio_Tiempo_Filt_Pro_Der: Error en cálculo - All numbers are identical in kruskal

Cambio_Tiempo_Filt_Con_Izq:
   Estadístico

## 5. Resumen de Variables Significativas CT_Filtrado

In [7]:
print("="*70)
print("RESUMEN: VARIABLES CT_FILTRADO SIGNIFICATIVAS")
print("="*70)

for Nombre_df in dfs_Finales.keys():
    print(f"\n📊 {Nombre_df}:")
    
    Variables_Sig = Todas_Variables_Significativas['Cambios_Tiempo_Filtrados_Significativos'][Nombre_df]
    
    if Variables_Sig:
        print(f"   Total: {len(Variables_Sig)} de {len(Variables_CT_Filtrado)} variables significativas")
        print(f"\n   Variables significativas:")
        for var in Variables_Sig:
            print(f"   ✅ {var}")
        
        # Mostrar las NO significativas
        No_Sig = [v for v in Variables_CT_Filtrado if v not in Variables_Sig]
        if No_Sig:
            print(f"\n   Variables NO significativas:")
            for var in No_Sig:
                print(f"   ❌ {var}")
    else:
        print("   ❌ Ninguna variable CT_Filtrado fue significativa")

print("\n" + "="*70)

RESUMEN: VARIABLES CT_FILTRADO SIGNIFICATIVAS

📊 Generales:
   Total: 2 de 4 variables significativas

   Variables significativas:
   ✅ Cambio_Tiempo_Filt_Con_Der
   ✅ Cambio_Tiempo_Filt_Pro_Der

   Variables NO significativas:
   ❌ Cambio_Tiempo_Filt_Pro_Izq
   ❌ Cambio_Tiempo_Filt_Con_Izq

📊 Ballotage:
   ❌ Ninguna variable CT_Filtrado fue significativa



## 6. Estadísticas Descriptivas por Categoría

In [8]:
def Calcular_Stats_Por_Categoria(df, variable, categorias):
    """
    Calcula estadísticas descriptivas por categoría.
    """
    stats_lista = []
    
    for categoria in categorias:
        datos = df[df['Categoria_PASO_2023'] == categoria][variable].dropna()
        
        if len(datos) > 0:
            stats_lista.append({
                'Categoria': categoria,
                'n': len(datos),
                'Media': datos.mean(),
                'Mediana': datos.median(),
                'DE': datos.std(),
                'Min': datos.min(),
                'Max': datos.max()
            })
        else:
            stats_lista.append({
                'Categoria': categoria,
                'n': 0,
                'Media': np.nan,
                'Mediana': np.nan,
                'DE': np.nan,
                'Min': np.nan,
                'Max': np.nan
            })
    
    return pd.DataFrame(stats_lista)


print("="*70)
print("ESTADÍSTICAS DESCRIPTIVAS POR CATEGORÍA")
print("="*70)

Categorias_Validas = [
    'Left_Wing', 'Progressivism', 'Centre',
    'Moderate_Right_A', 'Moderate_Right_B', 'Right_Wing_Libertarian'
]

for Nombre_df, df in dfs_Finales.items():
    print(f"\n📊 Dataset: {Nombre_df}")
    print("\n" + "-"*70)
    
    for var in Variables_CT_Filtrado:
        if var in df.columns:
            print(f"\n{var}:")
            
            df_stats = Calcular_Stats_Por_Categoria(df, var, Categorias_Validas)
            print(df_stats.to_string(index=False))
        else:
            print(f"\n{var}: Variable no encontrada")
    
    print("\n" + "-"*70)

ESTADÍSTICAS DESCRIPTIVAS POR CATEGORÍA

📊 Dataset: Generales

----------------------------------------------------------------------

Cambio_Tiempo_Filt_Pro_Izq:
             Categoria    n     Media  Mediana       DE     Min     Max
             Left_Wing  425 -0.260002      0.0 7.379468 -17.585 127.778
         Progressivism 1257 -0.800909      0.0 5.105300 -87.751  45.262
                Centre   84 -0.684667      0.0 2.301238 -13.000   7.540
      Moderate_Right_A  208 -1.657120      0.0 7.270047 -63.881  56.165
      Moderate_Right_B  212 -1.069182      0.0 5.190768 -44.497  14.666
Right_Wing_Libertarian  185 -1.668092      0.0 5.668968 -51.405  10.178

Cambio_Tiempo_Filt_Pro_Der:
             Categoria    n     Media  Mediana        DE      Min      Max
             Left_Wing  425 -0.464478   0.0000  7.595979  -54.611  44.2460
         Progressivism 1257 -1.499746   0.0000 11.976141 -204.486 107.4910
                Centre   84 -3.955208  -0.6630 20.818602 -181.307  22.0355
    

## 7. Comparación con CO_Filtrado

In [9]:
print("="*70)
print("COMPARACIÓN: CT_FILTRADO vs CO_FILTRADO")
print("="*70)

# Variables CO_Filtrado
Variables_CO_Filtrado = [
    'Cambio_Op_Filt_Pro_Izq',
    'Cambio_Op_Filt_Pro_Der',
    'Cambio_Op_Filt_Con_Izq',
    'Cambio_Op_Filt_Con_Der'
]

for Nombre_df in dfs_Finales.keys():
    print(f"\n📊 {Nombre_df}:")
    
    # CT_Filtrado significativas
    CT_Sig = Todas_Variables_Significativas.get('Cambios_Tiempo_Filtrados_Significativos', {}).get(Nombre_df, [])
    
    # CO_Filtrado significativas
    CO_Sig = Todas_Variables_Significativas.get('Cambios_Opinion_Filtrados_Significativos', {}).get(Nombre_df, [])
    
    print(f"\n   CT_Filtrado: {len(CT_Sig)}/4 variables significativas")
    print(f"   CO_Filtrado: {len(CO_Sig)}/4 variables significativas")
    
    print(f"\n   Comparación por tipo:")
    
    tipos = [
        ('Pro_Izq', 'Cambio_Tiempo_Filt_Pro_Izq', 'Cambio_Op_Filt_Pro_Izq'),
        ('Pro_Der', 'Cambio_Tiempo_Filt_Pro_Der', 'Cambio_Op_Filt_Pro_Der'),
        ('Con_Izq', 'Cambio_Tiempo_Filt_Con_Izq', 'Cambio_Op_Filt_Con_Izq'),
        ('Con_Der', 'Cambio_Tiempo_Filt_Con_Der', 'Cambio_Op_Filt_Con_Der')
    ]
    
    for tipo, var_ct, var_co in tipos:
        ct_sig = '✅' if var_ct in CT_Sig else '❌'
        co_sig = '✅' if var_co in CO_Sig else '❌'
        print(f"   {tipo:10} - CT: {ct_sig}  CO: {co_sig}")

print("\n" + "="*70)

COMPARACIÓN: CT_FILTRADO vs CO_FILTRADO

📊 Generales:

   CT_Filtrado: 2/4 variables significativas
   CO_Filtrado: 0/4 variables significativas

   Comparación por tipo:
   Pro_Izq    - CT: ❌  CO: ❌
   Pro_Der    - CT: ✅  CO: ❌
   Con_Izq    - CT: ❌  CO: ❌
   Con_Der    - CT: ✅  CO: ❌

📊 Ballotage:

   CT_Filtrado: 0/4 variables significativas
   CO_Filtrado: 0/4 variables significativas

   Comparación por tipo:
   Pro_Izq    - CT: ❌  CO: ❌
   Pro_Der    - CT: ❌  CO: ❌
   Con_Izq    - CT: ❌  CO: ❌
   Con_Der    - CT: ❌  CO: ❌



## 8. Funciones Auxiliares para Excel

In [10]:
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return hex_color.upper()

def aplicar_borde_completo(cell):
    thin_border = Border(
        left=Side(style='thin', color='000000'),
        right=Side(style='thin', color='000000'),
        top=Side(style='thin', color='000000'),
        bottom=Side(style='thin', color='000000')
    )
    cell.border = thin_border

Mapa_Colores_Categorias = {
    'Left_Wing': '#f65058',
    'Progressivism': '#0078bf',
    'Centre': '#009cdd',
    'Moderate_Right_A': '#f7d117',
    'Moderate_Right_B': '#f7d117',
    'Right_Wing_Libertarian': '#753bbd'
}

Etiquetas_Categorias = {
    'Left_Wing': 'Left Wing',
    'Progressivism': 'Progressivism',
    'Centre': 'Centre',
    'Moderate_Right_A': 'Moderate Right A',
    'Moderate_Right_B': 'Moderate Right B',
    'Right_Wing_Libertarian': 'Right Wing Libertarian'
}

## 9. Generar Tabla Excel Resumen

In [11]:
def Crear_Tabla_Excel_Resumen_CT_Filtrado(
    dfs_finales,
    variables,
    categorias,
    nombre_archivo='Resumen_CT_Filtrado.xlsx',
    carpeta_destino='Tablas_Excel_Resumen'
):
    """
    Crea tabla Excel resumen de CT_Filtrado.
    """
    
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    wb = Workbook()
    
    # Una hoja por dataset
    for idx, (nombre_df, df) in enumerate(dfs_finales.items()):
        if idx == 0:
            ws = wb.active
            ws.title = nombre_df[:31]  # Max 31 caracteres
        else:
            ws = wb.create_sheet(title=nombre_df[:31])
        
        fila = 1
        
        # TÍTULO
        ws.merge_cells(start_row=fila, start_column=1, end_row=fila, end_column=8)
        cell = ws.cell(fila, 1)
        cell.value = f'Resumen CT_Filtrado - {nombre_df}'
        cell.font = Font(bold=True, size=14, color='FFFFFF')
        cell.fill = PatternFill(start_color='2C5282', end_color='2C5282', fill_type='solid')
        cell.alignment = Alignment(horizontal='center', vertical='center')
        
        fila += 2
        
        # ENCABEZADO
        headers = ['Variable', 'Categoría', 'n', 'Media', 'DE', 'Mediana', 'Min', 'Max']
        for col_idx, header in enumerate(headers, start=1):
            cell = ws.cell(fila, col_idx)
            cell.value = header
            cell.font = Font(bold=True, color='FFFFFF', size=10)
            cell.fill = PatternFill(start_color='5B9BD5', end_color='5B9BD5', fill_type='solid')
            cell.alignment = Alignment(horizontal='center', vertical='center')
        
        fila += 1
        
        # DATOS
        for var in variables:
            if var in df.columns:
                df_stats = Calcular_Stats_Por_Categoria(df, var, categorias)
                
                for idx_cat, row_data in df_stats.iterrows():
                    categoria = row_data['Categoria']
                    color_cat = hex_to_rgb(Mapa_Colores_Categorias.get(categoria, '#E7E6E6'))
                    etiqueta = Etiquetas_Categorias.get(categoria, categoria)
                    
                    # Variable
                    cell = ws.cell(fila, 1)
                    cell.value = var if idx_cat == 0 else ''
                    cell.alignment = Alignment(horizontal='left', vertical='center')
                    if idx_cat == 0:
                        cell.font = Font(bold=True)
                    
                    # Categoría
                    cell = ws.cell(fila, 2)
                    cell.value = etiqueta
                    cell.fill = PatternFill(start_color=color_cat, end_color=color_cat, fill_type='solid')
                    cell.font = Font(bold=True, color='FFFFFF')
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                    
                    # n
                    cell = ws.cell(fila, 3)
                    cell.value = int(row_data['n'])
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                    
                    # Media
                    cell = ws.cell(fila, 4)
                    if not np.isnan(row_data['Media']):
                        cell.value = f"{row_data['Media']:.3f}"
                    else:
                        cell.value = '—'
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                    
                    # DE
                    cell = ws.cell(fila, 5)
                    if not np.isnan(row_data['DE']):
                        cell.value = f"{row_data['DE']:.3f}"
                    else:
                        cell.value = '—'
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                    
                    # Mediana
                    cell = ws.cell(fila, 6)
                    if not np.isnan(row_data['Mediana']):
                        cell.value = f"{row_data['Mediana']:.3f}"
                    else:
                        cell.value = '—'
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                    
                    # Min
                    cell = ws.cell(fila, 7)
                    if not np.isnan(row_data['Min']):
                        cell.value = f"{row_data['Min']:.3f}"
                    else:
                        cell.value = '—'
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                    
                    # Max
                    cell = ws.cell(fila, 8)
                    if not np.isnan(row_data['Max']):
                        cell.value = f"{row_data['Max']:.3f}"
                    else:
                        cell.value = '—'
                    cell.alignment = Alignment(horizontal='center', vertical='center')
                    
                    fila += 1
                
                # Fila vacía entre variables
                fila += 1
        
        # Aplicar bordes
        for row in ws.iter_rows(min_row=1, max_row=fila-1, min_col=1, max_col=8):
            for cell in row:
                aplicar_borde_completo(cell)
        
        # Ajustar anchos
        ws.column_dimensions['A'].width = 30
        ws.column_dimensions['B'].width = 20
        for col in ['C', 'D', 'E', 'F', 'G', 'H']:
            ws.column_dimensions[col].width = 12
    
    # Guardar
    ruta_completa = os.path.join(carpeta_destino, nombre_archivo)
    wb.save(ruta_completa)
    
    print(f"\n✅ Tabla Excel guardada en: {ruta_completa}")
    
    return ruta_completa


# Generar tabla
print("Generando tabla Excel resumen...\n")

Crear_Tabla_Excel_Resumen_CT_Filtrado(
    dfs_finales=dfs_Finales,
    variables=Variables_CT_Filtrado,
    categorias=Categorias_Validas
)

print("\n✅ Tabla resumen generada exitosamente")

Generando tabla Excel resumen...


✅ Tabla Excel guardada en: Tablas_Excel_Resumen\Resumen_CT_Filtrado.xlsx

✅ Tabla resumen generada exitosamente


## 10. Resumen Final

In [12]:
print("="*70)
print("RESUMEN FINAL: CT_FILTRADO")
print("="*70)

for Nombre_df in dfs_Finales.keys():
    print(f"\n📊 {Nombre_df}:")
    
    # Ítems CT individuales significativos
    Items_CT_Sig = Todas_Variables_Significativas.get('Cambios_Tiempo_Por_Item_Significativos', {}).get(Nombre_df, [])
    print(f"\n   1. Ítems CT individuales significativos: {len(Items_CT_Sig)}")
    
    # Variables CT_Filtrado significativas
    Vars_CT_Filt_Sig = Todas_Variables_Significativas.get('Cambios_Tiempo_Filtrados_Significativos', {}).get(Nombre_df, [])
    print(f"   2. Variables CT_Filtrado significativas: {len(Vars_CT_Filt_Sig)}/4")
    
    if Vars_CT_Filt_Sig:
        for var in Vars_CT_Filt_Sig:
            print(f"      ✅ {var}")
    else:
        print(f"      ❌ Ninguna variable CT_Filtrado fue significativa")
    
    # Comparación con CO
    Vars_CO_Filt_Sig = Todas_Variables_Significativas.get('Cambios_Opinion_Filtrados_Significativos', {}).get(Nombre_df, [])
    print(f"\n   3. Comparación con CO_Filtrado:")
    print(f"      - CT_Filtrado significativas: {len(Vars_CT_Filt_Sig)}/4")
    print(f"      - CO_Filtrado significativas: {len(Vars_CO_Filt_Sig)}/4")

print(f"\n📁 Archivo generado:")
print(f"   - Resumen_CT_Filtrado.xlsx")

print("\n" + "="*70)
print("✅ ANÁLISIS COMPLETADO")
print("="*70)

RESUMEN FINAL: CT_FILTRADO

📊 Generales:

   1. Ítems CT individuales significativos: 14
   2. Variables CT_Filtrado significativas: 2/4
      ✅ Cambio_Tiempo_Filt_Con_Der
      ✅ Cambio_Tiempo_Filt_Pro_Der

   3. Comparación con CO_Filtrado:
      - CT_Filtrado significativas: 2/4
      - CO_Filtrado significativas: 0/4

📊 Ballotage:

   1. Ítems CT individuales significativos: 5
   2. Variables CT_Filtrado significativas: 0/4
      ❌ Ninguna variable CT_Filtrado fue significativa

   3. Comparación con CO_Filtrado:
      - CT_Filtrado significativas: 0/4
      - CO_Filtrado significativas: 0/4

📁 Archivo generado:
   - Resumen_CT_Filtrado.xlsx

✅ ANÁLISIS COMPLETADO
